업종별 수익률 대비 변동성을 시각화 하여, 수익률 대비 변동성이 낮은 안정적인 종목의 인사이트를 제공하는 영역

In [1]:
# 데이터 분석을 위한 패키지
import numpy as np
import pandas as pd

In [2]:
# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import FinanceDataReader as fdr

In [3]:
import koreanize_matplotlib

In [4]:
import datetime
from dateutil.relativedelta import relativedelta

In [5]:
from data_loader import load_sector_data

In [6]:
# 모든 컬럼,행을 출력하도록 설정
pd.set_option('display.max_columns', None) # None으로 설정하면 모든 컬럼 출력
pd.set_option('display.max_rows', None) #None으로 설정하면 모든 행 출력

In [7]:
# 회귀선의 의미 : 동일한 평균수익률을 가진 다른 업종에 비해 변동성이 적어 리스크가 낮다는 인사이트를 제공할 수 있음음


def analyze_sector_return_volatility(market=None, month_ago=3):
    # 데이터 로드 (임의 데이터 예제)
    df = load_sector_data(market, month_ago=month_ago)  # 사용자 정의 데이터 로드 함수

    # 시장의 평균 수익률 계산
    market_avg_return = df['TotalReturn'].mean()

    # 업종별 평균 수익률과 변동성 계산
    mean_return_by_sector = df.groupby('Sector').agg({
        'TotalReturn': 'mean',
        'Volatility': 'mean'
    }).reset_index().sort_values(by='TotalReturn', ascending=True)

    # Plotly로 회귀선 데이터를 계산
    fig = px.scatter(
        mean_return_by_sector,
        x="TotalReturn",
        y="Volatility",
        trendline="ols"
    )
    
    # 회귀선 데이터를 추출
    
    # 이전에 생성한 fig 객체에서 회귀선의 결과를 가져옴 회귀선의 모델 결과를 포함하는 객체를 가져옵니다. 이 객체에는 회귀선의 절편(intercept)과 기울기(slope) 등의 정보가 포함되어 있음
    trendline_data = px.get_trendline_results(fig).iloc[0].px_fit_results
    
    # trendline_data.params: 회귀선의 파라미터(절편과 기울기)를 반환합니다. 여기서 intercept는 회귀선의 y절편을 의미하고, slope는 회귀선의 기울기를 나타냅니다.
    intercept, slope = trendline_data.params                                

    # 회귀선 아래/위 판별 및 색상 추가
    mean_return_by_sector['Color'] = mean_return_by_sector.apply(
        # 현재 행의 변동성이 회귀선에서 예측한 변동성보다 낮은지를 확인, 회귀선의 방정식은 y=mx+b 형태이므로, 여기서 intercept는 y절편, slope는 기울기, row["TotalReturn"]는 x값
        lambda row: 'Below' if row['Volatility'] < (intercept + slope * row['TotalReturn']) \
                                            else 'Above',
        axis=1
    )
    
    # 회귀선 대비 아래에 있는 종목 수익률 내림차순으로 정렬한 df
    below_regression_line_df = mean_return_by_sector[mean_return_by_sector['Color'] == 'Below'].sort_values(by='TotalReturn', ascending=False)

    # 최종 Plotly 그래프 생성
    fig = px.scatter(
        mean_return_by_sector,
        x="TotalReturn",
        y="Volatility",
        color="Color",  # 회귀선 기준 색상 설정
        hover_data={"Sector": True},
        title="업종별 수익률 대비 변동성 지수",
        labels={"TotalReturn": "평균 수익률", "Volatility": "평균 변동성"},
        template="plotly_white"
    )

    # 회귀선 다시 추가
    fig.add_trace(go.Scatter(
        x=mean_return_by_sector["TotalReturn"],
        y=intercept + slope * mean_return_by_sector["TotalReturn"],
        mode="lines",
        name="회귀선(해당 시장의 수익률 대비 평균 변동성)",
        line=dict(color="black", dash="dash")
    ))
    
    # 제목 가운데 정렬렬
    fig.update_layout(
        title_x=0.5,
        title_font=dict(size=20, color='black', family='Arial')
        ) 
    
    fig.show()
    
    # 변동성 대비 수익률이 좋은 종목 df 반환
    display(below_regression_line_df)

In [8]:
analyze_sector_return_volatility('KOSPI')

3개월 간의 data를 불러옵니다.


,Sector,TotalReturn,Volatility,Color
36,소프트웨어,14.075000,2.150000,Below
59,창업투자,7.460000,2.320000,Below
40,양방향미디어와서비스,6.373333,2.806667,Below
20,무선통신서비스,5.805000,1.405000,Below
3,가정용기기와용품,1.124000,2.088000,Below
26,복합기업,0.745417,2.157917,Below
28,부동산,-0.548929,1.842143,Below
45,음료,-1.340000,1.820000,Below
44,은행,-1.722000,2.115000,Below
2,가스유틸리티,-2.140000,1.565455,Below


In [9]:
analyze_sector_return_volatility('KOSDAQ')

3개월 간의 data를 불러옵니다.


,Sector,TotalReturn,Volatility,Color
63,항공화물운송과물류,36.505000,4.785000,Below
40,운송인프라,17.980000,3.270000,Below
34,손해보험,13.430000,3.200000,Below
18,도로와철도운송,2.550000,1.300000,Below
36,식품과기본식료품소매,1.148000,3.240000,Below
66,"호텔,레스토랑,레저",0.971111,3.070000,Below
1,가구,0.776000,2.734000,Below
28,상업서비스와공급품,0.580952,2.632857,Below
49,전문소매,0.487500,2.005000,Below
20,디스플레이패널,0.380000,0.690000,Below


In [10]:
analyze_sector_return_volatility('ETF')

3개월 간의 data를 불러옵니다.


,Sector,TotalReturn,Volatility,Color
3,기타,2.688228,0.667975,Below
5,채권,1.573230,0.302981,Below
0,국내 시장지수,-6.233523,1.282955,Below
